# Identifiability of Structural Growth Model in Absence of Compound

The tumour growth inhibiting effects of Erlotinib and Gefitinib were modelled with a population PKPD model in [1]. A population PKPD model is a hierarchical model which consists of a structural model, a population model, and an error model. Each sub-model captures a different aspect of the tumour growth inhibition biology, and is parametrised by a set of parameters. 

In this notebook we explore the identifiability of the structural model reported in [1] in absence of the compound. In particular we investigate the effect of log-transforms and non-dimensionalisation of the model parameters on the stability of the inference. 

## Structural growth model in absence of the compound

In a PKPD model the structural model is a mechanistic or empirical description of the drug's pharmacokinetics and pharmacodynamics. However, a good PKPD model also needs to capture the biology well in absence of the drug, in order to predict it's effects properly. In this study this means that we need to have a good understanding of the tumour growth in absence of the compund. In [1] a structural model is proposed, which assumes that the tumour grows exponentially for small tumour volumes, and linearly for large tumour volumes

\begin{equation*}
\frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1}.
\end{equation*}

Here, $V^s_T$ is the predicted tumour volume by the structural model, $\lambda _0$ is the exponential growth rate, and $\lambda _1$ is the linear growth rate.

## Identifiability

The problem of the identifiability of non-linear ordinary differential equation (ODE) models is a multifacetted problem and has been addressed many times in the literature, e.g. [2, 3, 4].

We are interested in the parameters $V_0$, $\lambda _0$ and $\lambda _1$ that best describe the observed tumour growth in untreated mice, see the dedicated [notebook](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/control_growth/data_preparation.ipynb) for details on the data. Here $V_0$ is the initial tumour volume at time $t=0$ according to the structural model, i.e. $V^s_T(t=0)=V_0$. In order to find good parameters we will use Bayesian inference. Bayesian inference constructs distributions for the parameters $V_0$, $\lambda _0$ and $\lambda _1$, indicating the probability with which certain parameter values have generated the data.

In this notebook we want to make sure that 

- identifiability: technical challenges versus theoretical problems
- discuss reasons for non-dimensionalisation
    - in theory Bayesian inference works for any model
    - in practice performance is a lot improved by by having parameters of equal scale (most optimisation have only one step size)
    - avoid hard boundaries, discuss why
- model can be such that it's overparametrised
- or data does not provide enough information about data

## Implementation of naïve pooled model

We are using [myokit](http://myokit.org/) for the implementation of the structural model. Myokit enables us to solve the structural model ODE with an adaptive numerical solver called CVODE [3]. To implement the error model and perform the inference we are using [pints](https://pints.readthedocs.io/). 

Note that in general the quality of the inference of $\psi $ and $\theta _V$ can be significantly improved when all parameters are appropriately transformed. We will however choose to not transform the parameetrs at first, to illustrate how the inference may be stabilised with transformations.

### Implementation of structural model in myokit

We have implemented the structural model in myokit with untransformed parameters in a separate [module](https://github.com/DavAug/ErlotinibGefitinib/blob/master/pkpd/model.py). The structural model can now be created by calling ```pkpd.model.create_tumour_growth_model()```.

In [1]:
#
# Implementing the structural model in myokit.
#

from pkpd import model as m


# Create model
model = m.create_tumour_growth_model()

# Print structural model
print(model.code())

[[model]]
# Initial values
central.volume_t = 0

[central]
lambda_0 = 0
    in [1/day]
lambda_1 = 1
    in [cm^3/day]
time = 0 bind time
    in [day]
dot(volume_t) = 2 * (lambda_0 * (lambda_1 * volume_t)) / (2 * (lambda_0 * volume_t) + lambda_1)
    in [cm^3]




### Implementation of error model 
We use pints to implement the error model indirectly by defining a log-likelihood for the model parameters $\psi $ and $\theta _V$

\begin{equation*}
L(\psi , \theta _V | V^{\text{obs}}_T) = \mathbb{P}(V_T | \psi , \theta _V) \Big | _{V_T = V^{\text{obs}}_T} .
\end{equation*}

Note that here the likelihood is mathematically equivalent to the above defined distribution of tumour growth curves.

To use the structural model in pints we first need to wrap the myokit model by a `pints.ForwardModel`.

In [2]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial V^s_T, lambda_0, lambda_1
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, lambda_0, lambda_1 = parameters

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.lambda_0', lambda_0)
        self.sim.set_constant('central.lambda_1', lambda_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

We also need to import the tumour growth data that we cleaned in a previous [notebook](https://github.com/DavAug/ErlotinibGefitinib/blob/master/notebooks/control_growth/data_preparation.ipynb) to define the likelihood of the model parameters.

In [3]:
#
# Load cleaned LXF and VXF data sets.
#

import pandas as pd


# Get path of current working directory
path = os.getcwd()

# Read cleaned LXF A677 control growth data
lxf_data = pd.read_csv(path + '/data/lxf_control_growth.csv')

# Read cleaned VXF A341 control growth data
vxf_data = pd.read_csv(path + '/data/vxf_control_growth.csv')

With a `pints.ForwardModel` implementing the structural model for the tumour growth and the time-series data of the control tumour growth, we can now construct the likelihood for the model parameters $\psi $ and $\theta $.

(Note that we still have to create likelihoods for the individual mice, because myokit does not allow multiple measurements at the same time point.)

### Likelihood for LXF A677 tumour growth model parameters

In [4]:
#
# Construct likelihood for LXF A677 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = lxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = lxf_data['#ID'] == ids

    # Get relevant time points
    times = lxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = lxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
lxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)

### Likelihood for VXF A341 tumour growth model parameters

In [5]:
#
# Construct likelihood for VXF A341 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = vxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = vxf_data['#ID'] == ids

    # Get relevant time points
    times = vxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = vxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
vxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)

## Identifiability check

some explanation

In [6]:
#
# Check identifiability of LXF A677 problem.
#

import pints


# Get number of optimisation runs and number of parameters
n = 5
n_parameters = lxf_log_likelihood.n_parameters()

# Initial guess of parameters
parameters = np.array([
    [10E-2, 10E-2, 10E-2, 10E-2],
    [10E-1, 10E-1, 10E-1, 10E-1],
    [10E0, 10E0, 10E0, 10E0],
    [10E1, 10E1, 10E1, 10E1],
    [10E2, 10E2, 10E2, 10E2]])

# Create estimate container
estimates = np.empty(shape=(n, n_parameters))

print('Running...')
# estimate parameters
for i in range(n):
    print('Round %d' % (i+1))

    # Initial guess of parameters
    params = parameters[i, :]

    # Choose optimisation method
    optimiser = pints.CMAES

    # Create optimisation object
    opt = pints.OptimisationController(
        function=lxf_log_likelihood,
        x0=params,
        method=optimiser)

    # Disable logging mode
    opt.set_log_to_screen(False)

    # Parallelise optimisation
    opt.set_parallel(True)

    # Optimise likelihood
    est, _ = opt.run()

    # Save estimates
    estimates[i, :] = est

print('Done!')

print(' ')
print('Estimates: ')
print('Initial tumour volume [cm^3]: ', estimates[:, 0])
print('Exponential growth rate \lambda _0 [1/day]: ', estimates[:, 1])
print('Linear growth rate \lambda _1 [cm^3/day]: ', estimates[:, 2])
print('Standard deviation of base-level noise [cm^3]: ', estimates[:, 3])

Running...
Round 1
Round 2
Round 3
Round 4
Round 5
Done!
 
Estimates: 
Initial tumour volume [cm^3]:  [1.34796622 0.93024172 1.34753463 1.34111851 1.27354206]
Exponential growth rate \lambda _0 [1/day]:  [ 3.66072941e+02 -2.50009666e-01  1.82867188e+02 -1.01997319e+01
 -3.28346683e-02]
Linear growth rate \lambda _1 [cm^3/day]:  [  0.3587859    0.31194863   0.35883914   0.35750309 144.84249923]
Standard-deviation of base-level noise [cm^3]:  [4.51403325 4.50053914 4.51405733 4.51374813 8.34865767]


## Log-transform structural model parameters

In [7]:
#
# Define pints model wrapper with log-transformed parameters.
#

import myokit
import numpy as np
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial V^s_T, lambda_0, lambda_1
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, log_parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, lambda_0, lambda_1 = np.exp(log_parameters)

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.lambda_0', lambda_0)
        self.sim.set_constant('central.lambda_1', lambda_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [8]:
#
# Construct likelihood for LXF A677 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = lxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = lxf_data['#ID'] == ids

    # Get relevant time points
    times = lxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = lxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
lxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)

In [9]:
#
# Construct likelihood for VXF A341 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = vxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = vxf_data['#ID'] == ids

    # Get relevant time points
    times = vxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = vxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
vxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)

In [16]:
#
# Check identifiability of LXF A677 problem.
#

import numpy as np
import pints


# Get number of optimisation runs and number of parameters
n = 5
n_parameters = lxf_log_likelihood.n_parameters()

# Initial guess of parameters [initial volume, lambda_0, lambda_1, sigma]
log_parameters_and_sigma = np.array([0.0, 0.0, 0.0, 1.0])

# Standard deviatoin around initial guesses
sigma0 = np.array([0.5, 0.5, 0.5, 0.5])

# Create estimate container
estimates = np.empty(shape=(n, n_parameters))

print('Running...')
# estimate parameters
for i in range(n):
    print('Round %d' % (i+1))

    # Choose optimisation method
    optimiser = pints.CMAES

    # Create optimisation object
    opt = pints.OptimisationController(
        function=lxf_log_likelihood,
        x0=log_parameters_and_sigma,
        sigma0=sigma0,
        method=optimiser)

    # Disable logging mode
    opt.set_log_to_screen(False)

    # Parallelise optimisation
    opt.set_parallel(True)

    # Optimise likelihood
    log_est, _ = opt.run()

    # Save estimates (transform log-parameters, but keep sigma)
    estimates[i, :-1] = np.exp(log_est[:-1])
    estimates[i, -1] =log_est[-1]

print('Done!')

print(' ')
print('Estimates: ')
print('Initial tumour volume [cm^3]: ', estimates[:, 0])
print('Exponential growth rate \lambda _0 [1/day]: ', estimates[:, 1])
print('Linear growth rate \lambda _1 [cm^3/day]: ', estimates[:, 2])
print('Standard deviation of base-level noise [cm^3]: ', estimates[:, 3])

Running...
Round 1
Round 2
Round 3
Round 4
Round 5
Done!
 
Estimates: 
Initial tumour volume [cm^3]:  [1.34771772 1.34795312 1.34770143 1.34740049 1.34745836]
Exponential growth rate \lambda _0 [1/day]:  [2.03554194e+03 3.66076628e+02 4.82434343e+13 8.42074049e+02
 8.29368456e+02]
Linear growth rate \lambda _1 [cm^3/day]:  [0.35874319 0.35878604 0.358735   0.35877458 0.35877169]
Standard-deviation of base-level noise [cm^3]:  [4.51404352 4.51403982 4.51404453 4.51404477 4.51404398]


## Non-dimensionalise parameters

In [21]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#
# Input parameters are scaled to an expected scale and log-trasnformed to stabilise
# inference.
#

import myokit
import numpy as np
import pints

from pkpd import model as model


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

        # Characteristic scale [intial volume, lambda_0, lambda_1]
        self._char_scale = np.array([1.3, 3E02, 0.35])

    def n_parameters(self):
        """
        Number of parameters to fit. Here initial V^s_T, lambda_0, lambda_1
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, log_parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, lambda_0, lambda_1 = np.exp(log_parameters) * self._char_scale

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.lambda_0', lambda_0)
        self.sim.set_constant('central.lambda_1', lambda_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [22]:
#
# Construct likelihood for LXF A677 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = lxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = lxf_data['#ID'] == ids

    # Get relevant time points
    times = lxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = lxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
lxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)#
# Construct likelihood for LXF A677 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = lxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = lxf_data['#ID'] == ids

    # Get relevant time points
    times = lxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = lxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
lxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)

In [23]:
#
# Construct likelihood for VXF A341 tumour growth model parameters.
#


import numpy as np
import pandas as pd
import pints


# Create inverse problem for each mouse ID
mouse_ids = vxf_data['#ID'].unique()
log_likelihoods = []
for ids in mouse_ids:
    # Create mask for mouse with specfied ID
    mouse_mask = vxf_data['#ID'] == ids

    # Get relevant time points
    times = vxf_data[mouse_mask]['TIME in day'].to_numpy()

    # Get measured tumour volumes
    observed_volumes = vxf_data[mouse_mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create Gaussian log-likelihood TODO: Change to combined error model
    log_likelihoods.append(pints.GaussianLogLikelihood(problem))

# Create one log_likelihood for the inference from the individual problems
vxf_log_likelihood = pints.SumOfIndependentLogPDFs(log_likelihoods)

In [24]:
#
# Check identifiability of LXF A677 problem.
#

import numpy as np
import pints


# Get number of optimisation runs and number of parameters
n = 5
n_parameters = lxf_log_likelihood.n_parameters()

# Initial guess of parameters [initial volume, lambda_0, lambda_1, sigma]
log_parameters_and_sigma = np.array([0.0, 0.0, 0.0, 1.0])

# Standard deviatoin around initial guesses
sigma0 = np.array([0.5, 0.5, 0.5, 0.5])

# Create estimate container
estimates = np.empty(shape=(n, n_parameters))

print('Running...')
# estimate parameters
for i in range(n):
    print('Round %d' % (i+1))

    # Choose optimisation method
    optimiser = pints.CMAES

    # Create optimisation object
    opt = pints.OptimisationController(
        function=lxf_log_likelihood,
        x0=log_parameters_and_sigma,
        sigma0=sigma0,
        method=optimiser)

    # Disable logging mode
    opt.set_log_to_screen(False)

    # Parallelise optimisation
    opt.set_parallel(True)

    # Optimise likelihood
    est, _ = opt.run()

    # Transform parameters back and save estimates
    char_scale = np.array([1.3, 3E02, 0.35])  # Defined in PintsModel
    estimates[i, :-1] = np.exp(est[:-1]) * char_scale
    estimates[i, -1] = est[-1]

print('Done!')

print(' ')
print('Estimates: ')
print('Initial tumour volume [cm^3]: ', estimates[:, 0])
print('Exponential growth rate \lambda _0 [1/day]: ', estimates[:, 1])
print('Linear growth rate \lambda _1 [cm^3/day]: ', estimates[:, 2])
print('Standard deviation of base-level noise [cm^3]: ', estimates[:, 3])

Running...
Round 1
Round 2
Round 3
Round 4
Round 5
Done!
 
Estimates: 
Initial tumour volume [cm^3]:  [1.34797863 1.34796979 1.34746332 1.3475335  1.34739554]
Exponential growth rate \lambda _0 [1/day]:  [366.0679323  366.07135534 829.36515154 182.86743461 842.07710061]
Linear growth rate \lambda _1 [cm^3/day]:  [0.35878428 0.35878532 0.35877156 0.35883931 0.35877458]
Standard deviation of base-level noise [cm^3]:  [4.5140468  4.51403935 4.51404432 4.51405563 4.51404329]


## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)
- <a name="ref2"> [2] </a> Bellman, R. & Åström, K. On structural identifiability.Mathematical Biosciences7, 329 – 339 (1970)
- <a name="ref2"> [3] </a> Janzén, D. L. I.et al.Parameter identifiability of fundamental pharmacodynamic models.Frontiers in Physiology7, 590 (2016)
- <a name="ref2"> [4] </a> Lavielle, M. & Aarons, L. What do we mean by identifiability in mixed effects models?Journal of Pharmacoki-netics and Pharmacodynamics43, 111–122 (2016)